マスターブランチ  
https://github.com/maxpumperla/deep_learning_and_the_game_of_go

P73までのコードはモジュールとして実装  
dlgo/gotypes.py: データ型 Player, Pointを定義  
dlgo/goboard_slow.py: 囲碁のルール，着手(Move)，連(GoString)，打石のシステム(Board)を実装．自殺手と劫を禁じる  
dlgo/utils.py: 着手の内容と碁盤の状況出力(print_move, print_board)  
dlgo/agent/base.py: プレイヤーボットのインタフェース(Agent)  
dlgo/agent/helpers.py: 打石点が眼であるかを判定する関数(is_point_an_eye)を実装  
dlgo/agent/naive.py: 眼をつぶさず，禁じ手にならなければあとはランダムで打石する，最弱ボット(RandomBot)  

ボット同士で対局させる (bot_v_bot.pyを基に作成)

In [1]:
from dlgo import gotypes
from dlgo import goboard_slow as goboard
from dlgo.agent.naive import RandomBot
from dlgo.utils import print_board, print_move
import time
from IPython.display import clear_output

In [3]:
board_size = 9
game = goboard.GameState.new_game(board_size)
bots = {
    gotypes.Player.black: RandomBot(),
    gotypes.Player.white: RandomBot()
}

while not game.is_over():
    time.sleep(1.0) # ボットの着手が早すぎるので見えるよう調整
    
    # print(chr(27) + "[2J") # 着手する前に画面をクリア
    clear_output()
    
    print_board(game.board)
    bot_move = bots[game.next_player].select_move(game)
    print_move(game.next_player, bot_move)
    game = game.apply_move(bot_move)

print("GAME OVER!")

 9 xxxxxxooo
 8 xooxxoxox
 7 oxxoxoxxx
 6 ooooxoxxx
 5 ooxxxxxoo
 4 xxooooxoo
 3 ooooxxooo
 2 oxoooxoxx
 1 xxooxxoxx
   ABCDEFGHI
Player.black passes
GAME OVER!
